In [ ]:
%cd ..
%pwd

In [ ]:
from watertap3.utils import watertap_setup, get_case_study, run_water_tap, run_water_tap_ro, show_train2, get_fixed_onm_reduction, run_sensitivity, get_results_table
import pandas as pd

In [ ]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"

In [ ]:
for source_water in ['industrial']:#df.source_water_category.unique():
    for case_study in ['upw']:#df[df.source_water_category == source_water].case_study.unique(): 
        h1 = df[df.case_study == case_study]
        scenarios = h1.scenario.unique()
        for scenario in ['zld_a']: #scenarios: 

            print("------------------------------------------------------")
            print("------------------------------------------------------")
            print("-----------", case_study, scenario, "-----------------")
            print("------------------------------------------------------")
            print("------------------------------------------------------")

            desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

            source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

            skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
            skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
            ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()
#             ro_bounds = 'seawater'

            m = watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                                  scenario = scenario, source_scenario = "baseline")

            m = get_case_study(m=m)

            if "1p5" in scenarios:
                m.fs.irrigation_and_drainage.flow_vol_in.fix(0.0657)
            
            ############################################################
            ############################################################
            # set recovery for uranium case RO units.
            if case_study == "uranium":
                m.fs.ion_exchange.removal_fraction[0, "tds"].unfix()
                m.fs.ion_exchange.water_recovery.fix(0.967)
                m.fs.ion_exchange.anion_res_capacity.unfix()
                m.fs.ion_exchange.cation_res_capacity.unfix()
            
            if scenario == "swd_well":
                m.fs.produced_water_for_dw.conc_mass_in[0, "tds"].fix(50)
            
            # run and and return baseline with 0 degrees of freedom
            m = run_water_tap_ro(m, source_water_category = source_water_category, 
                                 desired_recovery = desired_recovery, skip_small = skip_small_base, 
                                 ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

            if scenario == "baseline":
#                 run_sensitivity(m = m, save_results = True, scenario = scenario, 
#                                     case_study = case_study, skip_small_sens = skip_small_sens)

                if source_water == "seawater":
                    m = get_fixed_onm_reduction(m, reduction_value_list = [0.9, 0.8, 0.7, 0.6, 0.5],
                                                  skip_small=skip_small_base)

                if case_study == "san_luis":
                    n = 1
                    for flow_scenario in [0.0657, 0.1314, 0.2191]:
                        m.fs.irrigation_and_drainage.flow_vol_in.fix(flow_scenario)
                        for key in m.fs.pfd_dict.keys():
                            if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                                getattr(m.fs, key).feed.pressure.unfix()
                                getattr(m.fs, key).membrane_area.unfix()

                        print("adjusted baseline flow -->", flow_scenario)
                        run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                        if n == 1: s_name = "1p5_mgd"
                        if n == 2: s_name = "3_mgd"
                        if n == 3: s_name = "5_mgd"
                        df_no2 = get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)                
                        n = n + 1
                        
            
            if case_study == "lithium":
                ### double cap
                if scenario == "baseline": 
                    s_name = "double_cap"
                else:
                    s_name = scenario + "_double_cap"
                print("goes in")
                flow_in_x2 = m.fs.lithium_brine.flow_vol_in[0]() * 2
                m.fs.lithium_brine.flow_vol_in.fix(flow_in_x2)
                print("adjusted baseline flow -->", flow_in_x2)
                run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                df_no3 = get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)

            
            dwi_list = ["emwd", "big_spring"]   
            if m.fs.train["case_study"] in dwi_list:
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "deep_well_injection":
                        run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)

            if scenario in ["edr_ph_ro", "ro_and_mf"]:
                run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)                

            if source_water == "power":
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "evaporation_pond":
                        run_sensitivity(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)  



In [9]:
m.fs.reverse_osmosis_2_stage.component_mass_balance.display()

component_mass_balance : Size=13
    Key                              : Lower : Body               : Upper
        (0.0, 'alkalinity_as_caco3') :   0.0 : 36.000000000000014 :   0.0
                    (0.0, 'calcium') :   0.0 : 36.000000000000014 :   0.0
                   (0.0, 'chloride') :   0.0 : 36.000000000000014 :   0.0
    (0.0, 'electrical_conductivity') :   0.0 : 36.000000000000014 :   0.0
                  (0.0, 'magnesium') :   0.0 : 36.000000000000014 :   0.0
        (0.0, 'nitrate_as_nitrogen') :   0.0 : 36.000000000000014 :   0.0
                  (0.0, 'potassium') :   0.0 : 36.000000000000014 :   0.0
                    (0.0, 'silicon') :   0.0 : 36.000000000000014 :   0.0
                     (0.0, 'sodium') :   0.0 : 36.000000000000014 :   0.0
                    (0.0, 'sulfate') :   0.0 : 36.000000000000014 :   0.0
                     (0.0, 'sulfur') :   0.0 : 36.000000000000014 :   0.0
                        (0.0, 'tds') :   0.0 : 36.000000000000014 :   0.0
     

In [ ]:
m.fs.display()

In [ ]:
m.fs.ro_second_stage.display()

In [ ]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

In [ ]:
m.fs.ro_second_pass.display()

In [ ]:
# case_study = 'ashkelon'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

# case_study = 'carlsbad'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

# case_study = 'santa_barbara'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

# case_study = 'tampa_bay'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'seawater'
# ro_bounds = 'seawater'
# desired_recovery = 1

case_study = 'emwd'
scenario = 'baseline'
reference = 'nawi'
source_water = 'emwd_ca_brackish'
ro_bounds = 'other'
desired_recovery = 1

# case_study = 'irwin'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'brackish'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'kbhdp'
# scenario = 'zld_c'
# reference = 'nawi'
# source_water = 'kbhdp_brackish_ave'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'big_spring'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'big_spring_feed'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'hrsd'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'hrsd_municipal'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'ocwd'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'ocwd_feed'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'solaire'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['solaire_graywater', 'solaire_blackwater']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'heap_leaching'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['ore_heap_make_up', 'ore_heap_recycle']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'lithium'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'lithium_brine'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'uranium'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['uranium_active', 'uranium_restore']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'cherokee'
# scenario = 'zld_brine_conc'
# reference = 'nawi'
# source_water = ['cherokee_ct_makeup', 'denver_city_water']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'gila_river'
# scenario = 'both_ro'
# reference = 'nawi'
# source_water = 'gila_river_water'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'beef'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'beef_processing'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'monterey_one'
# scenario = 'add_ix'
# reference = 'nawi'
# source_water = 'secondary_ww'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'san_luis'
# scenario = 'stage_1p5mgd_edr'
# reference = 'nawi'
# source_water = 'irrigation_and_drainage'
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'iron_and_steel'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['steel_oily_ww', 'steel_metal_ww', 'hot_mill_blowdown', 'hot_mill_blowdown_b']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'pulp_and_paper'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['main_sewer', 'corrosive_sewer']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'upw'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = ['fab25', 'upw_brine']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'kern_county'
# scenario = 'ro_and_mf'
# reference = 'nawi'
# source_water = ['produced_water', 'surface_water']
# ro_bounds = 'other'
# desired_recovery = 1

# case_study = 'damodar'
# scenario = 'baseline'
# reference = 'nawi'
# source_water = 'powder_river_basin_cbm'
# ro_bounds = 'other'
# desired_recovery = 1

m = watertap_setup(dynamic=False, case_study=case_study, reference=reference, 
                      scenario=scenario, source_scenario="baseline")

m = get_case_study(m=m)

m = run_water_tap_ro(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds,
                 source_water_category='other', scenario_name=scenario,
                 source_scenario='baseline')    

# run_water_tap(m, print_model_results='summary')